# 本セクションの目次
1. Avroフォーマット
2. 前方互換と後方互換と完全互換
3. メッセージキューとAvroを連携してみよう
4. Avroファイルの読み書き
5. Avroで前方互換をやってみよう

In [29]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [30]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

:: loading settings :: url = jar:file:/home/pyspark/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pyspark/.ivy2/cache
The jars for the packages stored in: /home/pyspark/.ivy2/jars
org.apache.spark#spark-streaming_2.13 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d68288ce-a580-4654-9cb6-d666e43d44f7;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in ce

# データラングリングとは？
データラングリングとは、データをこねくり回してデータをより使いやすくする作業のことを指します。

- 重複削除
- idから商品名を引っ張っってくる
- 使い物になる様に別テーブルとくっつける

データラングリングと呼ばれる対象は一般にはCSV、JSON、アクセスログもあるのですが、それ以外にもExcelのデータ、PDFのデータ
なども含まれています。

データラングリングというとかっこよく聞こえるかもしれないのですが、かなり地味な点と、エンジニアとしてラングリングを扱うには注意点がありますので  
その点について紹介をしていこうと思います。

データラングリングはPythonとPySparkを組み合わせながら進めていくことが多いです。

# テーブル形式を含むExcelのラングリング
Excelのラングリングは、Sparkで読み込みをすることができません。
そのためpandasを使ってExcelデータを読み込み、

In [4]:
import pandas as pd

df = pd.read_excel('./dataset/table_excel.xlsx')
print(df)



   hoge  peke
0   1.0   2.0
1   3.0   2.0


# テーブル形式を含まないExcelのラングリング

お次はテーブルっぽくないexcelのラングリングをしてみましょう。  
しかし心配は入りません。

Excelであればいつでも単純に処理をすることが可能です。


In [ ]:
import pandas as pd

df = pd.read_excel('./dataset/no_table_excel.xlsx')
print(df)

# PDFのラングリングを行ってみよう
PDFのラングリングは要注意です。
基本的にできることはできるのですが、出力したPDFの作り方によってはまともに読めないことがあります。

そのため、PDFのデータ解析をしたい！
という要望を受けたら、基本的には断りつつExcelに変更してもらうなどの対応をとる方が賢明です。

とはいえ、元のデータが残っておらずどうしてもやらなければならない時があるのでその時のために少しだけ方法を見てみましょう。
有効な方法は以下の２です。

- OCRでデータを読み取る(PyOCRなど)
- ガッツリデータを読み込む

今回はガッツリデータを読み込む方法で行ってみましょう。

In [8]:
from re import split
from pdfminer.high_level import extract_text
import re
import os
from decimal import Decimal

text = extract_text(os.path.join("./dataset", "no_table_pdf.pdf"))

lines=text.split('\n')

#空行削除
lines = list(filter(None, lines))

for line in lines:
    print(line)

売上
消費税
100
10
決済者
hogepeke



In [13]:
# あとは表示されたアウトプットをもとに整形をしていくだけです

dict={}
dict[lines[0]]=lines[2]
dict[lines[1]]=lines[3]
dict[lines[4]]=lines[5]

print(dict)



{'売上': '100', '消費税': '10', '決済者': 'hogepeke'}


In [28]:
pd_dict=pd.DataFrame.from_dict(dict,orient='index')

print(pd_dict)
print("-------------")
print(pd_dict.transpose())

            0
売上        100
消費税        10
決済者  hogepeke
-------------
    売上 消費税       決済者
0  100  10  hogepeke


In [33]:
sparkDF=spark.createDataFrame(pd_dict.transpose()) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- 売上: string (nullable = true)
 |-- 消費税: string (nullable = true)
 |-- 決済者: string (nullable = true)



+----+------+--------+
|売上|消費税|  決済者|
+----+------+--------+
| 100|    10|hogepeke|
+----+------+--------+



# データラングリングで気をつけること

ここまでみてどうだったでしょうか？
基本的にはできそうだけども、できなそう。。

というところだったかと思います。

In [ ]:
spark.stop()